In [1]:
import resources as r
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML, display
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from tqdm.notebook import tqdm_notebook
IPythonConsole.ipython_useSVG=True

layout = widgets.Layout(width='max-content')

display(HTML('''<style>
    .widget-label { min-width: 30ex !important; }
    .widget-inline-hbox { min-width: 50ex !important; }
    .widget-button { min-width: max-content }
</style>'''))

fluorophores, solvents, methods = r.fluorophores_solvents_methods()

# DF Modifications
def add_row_to_multiindex(df:pd.MultiIndex, row) -> None:
    for fluorophore in fluorophores:
        df.loc[(fluorophore , row), :] = None
        df.sort_index(inplace=True)

def make_multindex(fluorophores:list[r.Fluorophores], solvents:list[r.Solvents], methods:list[r.Methods], dfName:str) -> None:
    iterables = [fluorophores, methods]
    index = pd.MultiIndex.from_product(iterables, names=['fluorophore', 'method'])
    df = pd.DataFrame(np.full(((len(fluorophores)*len(methods)), len(solvents)), None), columns=solvents, index=index)
    df.to_pickle(f'resources/databases/fluorophores-ds/{dfName}')

def make_singleindex(fluorophores:list[r.Fluorophores], solvents:list[r.Solvents], dfName:str):
    iterables = [fluorophores, methods]
    df = pd.DataFrame(np.full((len(fluorophores), len(solvents)), None), columns=solvents, index=fluorophores)
    df.to_pickle(f'resources/databases/fluorophores-ds/{dfName}')

def make_multindex_multistate(fluorophores:list[r.Fluorophores], solvents:list[r.Solvents], states:list[r.States], dfName:str) -> None:
    iterables = [fluorophores, states]
    index = pd.MultiIndex.from_product(iterables, names=['fluorophore', 'states'])
    df = pd.DataFrame(np.full(((len(fluorophores)*len(methods)), len(states)), None), columns=solvents, index=index)
    df.to_pickle(f'resources/databases/fluorophores-ds/{dfName}')

def make_ds_spectra(fluorophores:list[r.Fluorophores], solvents:list[r.Solvents], spectra:list[r.spectraType], dsName:str) -> None:
    iterables = [fluorophores, spectra]
    index = pd.MultiIndex.from_product(iterables, names=['fluorophore', 'spectra'])
    df = pd.DataFrame(np.full(((len(fluorophores)*len(spectra)), len(solvents)), None), columns=solvents, index=index)
    df.astype(object)
    df.to_pickle(f'resources/databases/fluorophores-ds/{dsName}')

def make_ds_vals(fluorophores:list[r.Fluorophores], solvents:list[r.Solvents], labels:list[str], dsName:str) -> None:
    iterables = [fluorophores, labels]
    index = pd.MultiIndex.from_product(iterables, names=['fluorophore', 'energy'])
    df = pd.DataFrame(np.full(((len(fluorophores)*len(labels)), len(solvents)), None), columns=solvents, index=index)
    df.to_pickle(f'resources/databases/fluorophores-ds/{dsName}')

# Data Pullers
def check_s0(mon:r.monarchHandler) -> None:
    with out:
        fluorophores, solvents, methods = r.fluorophores_solvents_methods()
        with r.statusLoad(df='progress_s0') as df:
            with tqdm_notebook(total=(len(fluorophores)*len(methods)*len(solvents))) as pBar:
                for fluorophore in fluorophores: 
                    for method in methods:
                        for solvent in solvents:
                            pBar.set_description(f'{fluorophore.fluorophore}-{r.States.s0} in {solvent}: {method}')
                            pBar.update(1)
                            status = df.at[(fluorophore, method), solvent]
                            if status not in [r.Status.SMD.finished]:
                                job = r.Job(r.Software.orca, fluorophore, solvent, method, r.Basis.augccpvdz, r.PCM.smd, r.PCM.Eq.eq, r.States.s0, r.Jobs.opt)
                                orcaStatus = mon.checkJobStatus(job)
                                if orcaStatus == None:
                                    job = r.Job(r.Software.crest, fluorophore, solvent, r.Methods.crest, r.Basis.none, r.PCM.alpb, r.PCM.Eq.none, r.States.s0, r.Jobs.crest)
                                    crestStatus = mon.checkJobStatus(job)
                                    status = crestStatus if crestStatus != None else None
                                else:
                                    status = orcaStatus
                            df.at[(fluorophore, method), solvent] = status
                pBar.set_description(f's0: Done!')

def check_s1(mon:r.monarchHandler) -> None:
    with out:
        fluorophores, solvents, methods = r.fluorophores_solvents_methods()
        with r.statusLoad(df='progress_s1') as df:
            with tqdm_notebook(total=(len(fluorophores)*len(methods)*len(solvents))) as pBar:
                for fluorophore in fluorophores: 
                    for method in methods:
                        for solvent in solvents:
                            pBar.set_description(f'{fluorophore.fluorophore}-{r.States.s1} in {solvent}: {method}')
                            pBar.update(1)
                            status = df.at[(fluorophore, method), solvent]
                            if status not in [r.Status.SMD.finished]:
                                job = r.Job(r.Software.orca, fluorophore, solvent, method, r.Basis.augccpvdz, r.PCM.smd, r.PCM.Eq.eq, r.States.s1, r.Jobs.opt)
                                status = mon.checkJobStatus(job)
                            df.at[(fluorophore, method), solvent] = status
                pBar.set_description(f's1: Done!')

def check_s2(mon:r.monarchHandler) -> None:
    with out:
        fluorophores, solvents, methods = r.fluorophores_solvents_methods()
        fluorophores = [fluorophore for fluorophore in r.Fluorophores if (fluorophore.root == r.States.s2 and bool(fluorophore) == True)]
        with r.statusLoad(df='progress_s2') as df:
            with tqdm_notebook(total=(len(fluorophores)*len(methods)*len(solvents))) as pBar:
                for fluorophore in fluorophores: 
                    for method in methods:
                        for solvent in solvents:
                            pBar.set_description(f'{fluorophore.fluorophore}-{r.States.s1} in {solvent}: {method}')
                            pBar.update(1)
                            status = df.at[(fluorophore, method), solvent]
                            if status not in [r.Status.SMD.finished]:
                                job = r.Job(r.Software.orca, fluorophore, solvent, method, r.Basis.augccpvdz, r.PCM.smd, r.PCM.Eq.eq, r.States.s2, r.Jobs.opt)
                                status = mon.checkJobStatus(job)
                            df.at[(fluorophore, method), solvent] = status
                pBar.set_description(f's2: Done!')

def check_gas(mon:r.monarchHandler) -> None:
    global checkCASSCF
    with out:
        fluorophores, solvents, methods = r.fluorophores_solvents_methods()
        methods = [r.Methods.wb97xd, r.Methods.mp2]
        if checkCASSCF: methods += [r.Methods.casscf]
        with r.statusLoad('progress_gas') as df:
            with tqdm_notebook(total=(len(fluorophores)*len(methods)*2)+2) as pBar:
                with tqdm_notebook(total=((18-4)*3), display=False) as pBar2:
                    displayedPbar2 = False
                    for fluorophore in fluorophores:
                        if fluorophore == r.Fluorophores.az: states = [r.States.s0, r.States.s1, fluorophore.root]
                        else: states = [r.States.s0, r.States.s1]
                        for state in states:
                            for method in methods:
                                status = df.at[(fluorophore, method), state]
                                pBar.update(1)
                                if status != r.Status.SMD.finished:
                                    if method == r.Methods.wb97xd:
                                        pBar.set_description(f'{fluorophore.fluorophore}-{state} in {r.Solvents.gas}: {method}')
                                        job = r.Job(r.Software.orca, fluorophore, r.Solvents.gas, method, r.Basis.augccpvdz, r.PCM.none, r.PCM.Eq.none, state, r.Jobs.opt)
                                        status = mon.checkJobStatus(job)
                                    elif method == r.Methods.mp2:
                                        pBar.set_description(f'{fluorophore.fluorophore}-{state} in {r.Solvents.gas}: {method}')
                                        job = r.Job(r.Software.pyscf, fluorophore, r.Solvents.gas, method, r.Basis.augccpvdz, r.PCM.none, r.PCM.Eq.none, state, r.Jobs.mp2Natorb)
                                        status = mon.checkJobStatus(job)
                                    else:
                                        if not displayedPbar2:
                                            display(pBar2.container)
                                            displayedPbar2 = True

                                        pBar2.reset()
                                        if fluorophore.active == (0,0):
                                            casscf = (4,4)
                                            statusList = []
                                            status = None
                                            pBar.set_description(f'{fluorophore.fluorophore}-{state} in {r.Solvents.gas}: {method}')
                                            
                                            while casscf != (18,18):
                                                pBar2.update(1)
                                                pBar2.set_description(f'{fluorophore.fluorophore}-{state} in {r.Solvents.gas}: {method}{casscf}')
                                                nelec, norbs = casscf
                                                job = r.Job(r.Software.pyscf, fluorophore, r.Solvents.gas, method, r.Basis.def2tzvpd, r.PCM.none, r.PCM.Eq.none, state, r.Jobs.casscfOpt, casscf=casscf)
                                                status = mon.checkJobStatus(job)
                                                statusList += [status]

                                                pBar2.update(1)
                                                casscf = (nelec+1, norbs)
                                                pBar2.set_description(f'{fluorophore.fluorophore}-{state} in {r.Solvents.gas}: {method}{casscf}')
                                                job = r.Job(r.Software.pyscf, fluorophore, r.Solvents.gas, method, r.Basis.def2tzvpd, r.PCM.none, r.PCM.Eq.none, state, r.Jobs.casscfOpt, casscf=casscf)
                                                status = mon.checkJobStatus(job)
                                                statusList += [status]
                                                
                                                pBar2.update(1)
                                                casscf = (nelec, norbs+1)
                                                pBar2.set_description(f'{fluorophore.fluorophore}-{state} in {r.Solvents.gas}: {method}{casscf}')
                                                job = r.Job(r.Software.pyscf, fluorophore, r.Solvents.gas, method, r.Basis.def2tzvpd, r.PCM.none, r.PCM.Eq.none, state, r.Jobs.casscfOpt, casscf=casscf)
                                                status = mon.checkJobStatus(job)
                                                statusList += [status]
                                                casscf = (nelec+1, norbs+1)

                                            if r.Status.CASSCF.queued in statusList: status = r.Status.CASSCF.queued
                                            elif r.Status.CASSCF.running in statusList: status = r.Status.CASSCF.running
                                            elif r.Status.CASSCF.failed in statusList: status = r.Status.CASSCF.failed
                                            elif r.Status.CASSCF.finished in statusList: status = r.Status.CASSCF.finished
                                            else: status = None

                                        else:
                                            pBar.set_description(f'{fluorophore.fluorophore}-{state} in {r.Solvents.gas}: {method}{fluorophore.active}')
                                            job = r.Job(r.Software.pyscf, fluorophore, r.Solvents.gas, method, r.Basis.def2tzvpd, r.PCM.none, r.PCM.Eq.none, state, r.Jobs.casscfOpt, casscf=fluorophore.active)
                                            status = mon.checkJobStatus(job)

                                        if status == r.Status.CASSCF.finished and fluorophore.active != (0,0):
                                            status = 'finalised'


                                df.at[(fluorophore, method), state] = status
                pBar.set_description(f'Gas Phase: Done!')

# Visualisers
def visualise_ds() -> None:
    out.clear_output()
    with out:
        print('Dataset')
        mols = []
        nameList = []
        for fluorophore in r.Fluorophores:
            if fluorophore.revised:
                nameList += [fluorophore.fluorophore]
                mols += [Chem.MolFromSmiles(fluorophore.smiles)]
        display(Draw.MolsToGridImage(mols, legends=nameList, subImgSize=(400, 400)))
        print('\nReference Species')
        mols = []
        nameList = []
        for fluorophore in r.Fluorophores:
            if fluorophore.ref:
                nameList += [fluorophore.fluorophore]
                mols += [Chem.MolFromSmiles(fluorophore.smiles)]
        display(Draw.MolsToGridImage(mols, legends=nameList, subImgSize=(400, 400)))

def prettyDisplay() -> None:
    global spectra
    global energy
    global solvent
    global qy
    global gas
    global ds
    global s0
    global s1
    global s2
    with out:
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            if ds or spectra:
                with r.statusLoad('spectra') as df:
                    print('Spectra Database')
                    display(df.notnull().style.applymap(lambda x: 'color : blue' if x == True  else 'color : red'))
                    print('\n')

            if ds or qy:
                with r.statusLoad('qy_ref') as df:
                    print('Reference Spectra')
                    display(df.notnull().style.applymap(lambda x: 'color : blue' if x == True  else 'color : red'))
                    print('\n')

            if ds or energy:
                with r.statusLoad('dataset') as df:
                    display(df.style.applymap(lambda x: 'color : blue' if x != None  else 'color : red').format(precision=3))
                    print('\n')

            for i, j in [('gas', gas), ('s0', s0), ('s1', s1), ('s2', s2)]:
                if j:
                    print(f'{i} Dataset')
                    with r.statusLoad(f'progress_{i}') as df:
                        display(df.style.applymap(lambda x: 'color : red'     if x == r.Status.Crest.failed      else '')
                                        .applymap(lambda x: 'color : green'   if x == r.Status.Crest.finished    else '')
                                        .applymap(lambda x: 'color : orange'  if x == r.Status.Crest.running     else '')
                                        .applymap(lambda x: 'color : blue'    if x == r.Status.Crest.queued      else '')
                                        .applymap(lambda x: 'color : purple'  if x == r.Status.Crest.timed_out   else '')
                                        .applymap(lambda x: 'color : red'     if x == r.Status.CASSCF.failed     else '')
                                        .applymap(lambda x: 'color : teal'    if x == r.Status.CASSCF.finished   else '')
                                        .applymap(lambda x: 'color : orange'  if x == r.Status.CASSCF.running    else '')
                                        .applymap(lambda x: 'color : blue'    if x == r.Status.CASSCF.queued     else '')
                                        .applymap(lambda x: 'color : purple'  if x == r.Status.CASSCF.timed_out  else '')
                                        .applymap(lambda x: 'color : orange'  if x == r.Status.SMD.running       else '')
                                        .applymap(lambda x: 'color : red'     if x == r.Status.SMD.failed        else '')
                                        .applymap(lambda x: 'color : teal'    if x == r.Status.SMD.finished      else '')
                                        .applymap(lambda x: 'color : blue'    if x == r.Status.SMD.queued        else '')
                                        .applymap(lambda x: 'color : purple'  if x == r.Status.SMD.timed_out     else '')
                                        .applymap(lambda x: 'color : orange'  if x == r.Status.MP2.running       else '')
                                        .applymap(lambda x: 'color : red'     if x == r.Status.MP2.failed        else '')
                                        .applymap(lambda x: 'color : teal'    if x == r.Status.MP2.finished      else '')
                                        .applymap(lambda x: 'color : blue'    if x == r.Status.MP2.queued        else '')
                                        .applymap(lambda x: 'color : purple'  if x == r.Status.MP2.timed_out     else '')
                                        .applymap(lambda x: 'color : purple'  if x == 'finalised'                else '')
                                        .applymap(lambda x: 'color : black'   if x == None                       else ''))
                        print('\n')

def showDF() -> None:
    out.clear_output()
    prettyDisplay()

def showQYDF() -> None:
    global spectra
    global energy
    global solvent
    global qy
    global gas
    global ds
    global s0
    global s1
    global s2

    spectraTMP = spectra
    energyTMP = energy
    qyTMP = qy
    gasTMP = gas
    dsTMP = ds
    s0TMP = s0
    s1TMP = s1
    s2TMP = s2

    spectra = False
    energy = False
    qy = True
    gas = False
    ds = False
    s0 = False
    s1 = False
    s2 = False

    out.clear_output()
    prettyDisplay()

    spectra = spectraTMP
    energy = energyTMP
    qy = qyTMP
    gas = gasTMP
    ds = dsTMP
    s0 = s0TMP
    s1 = s1TMP
    s2 = s2TMP

def showALL() -> None:
    global spectra
    global energy
    global solvent
    global qy
    global gas
    global ds
    global s0
    global s1
    global s2

    spectraTMP = spectra
    energyTMP = energy
    qyTMP = qy
    gasTMP = gas
    dsTMP = ds
    s0TMP = s0
    s1TMP = s1
    s2TMP = s2

    spectra = True
    energy = True
    qy = True
    gas = True
    ds = True
    s0 = True
    s1 = True
    s2 = True

    out.clear_output()
    prettyDisplay()

    spectra = spectraTMP
    energy = energyTMP
    qy = qyTMP
    gas = gasTMP
    ds = dsTMP
    s0 = s0TMP
    s1 = s1TMP
    s2 = s2TMP

# Callable
def resetDF() -> None:
    out.clear_output()
    global displayDF
    global spectra
    global energy
    global qy
    global gas
    global s0
    global s1
    global s2
    if spectra:
        make_ds_spectra(fluorophores, solvents, [r.spectraType.absorbance, r.spectraType.emission, r.spectraType.excitation, r.spectraType.ftir, r.spectraType.qy, r.spectraType.lifetime], 'spectra')

    if energy:
        make_ds_vals(fluorophores, solvents, ['a', 'a_g', 'e', 'e_g', 'zz', 'zz_g', 'qy', 'fl', 'conc'], 'dataset')

    if qy: 
        make_ds_spectra([r.Fluorophores.cv, r.Fluorophores.rb], [r.Solvents.etoh, r.Solvents.h2o], [r.spectraType.qy], 'qy_ref')

    if gas:
        make_multindex(fluorophores, [r.States.s0, r.States.s1, r.States.s2], [r.Methods.wb97xd, r.Methods.casscf, r.Methods.mp2], 'progress_gas')

    if s0:
        make_multindex(fluorophores, solvents, methods, 'progress_s0')

    if s1:
        make_multindex(fluorophores, solvents, methods, 'progress_s1')

    if s2:
        make_multindex([r.Fluorophores.az], solvents, methods, 'progress_s2')

    if displayDF:
        prettyDisplay()
        
def addFunctional() -> None:
    out.clear_output()
    global displayDF
    global functional
    global spectra
    global energy
    global gas
    global s0
    global s1
    global s2

    dfList = []
    if gas: dfList += ['progress_gas']
    if s0:  dfList += ['progress_s0']
    if s1:  dfList += ['progress_s1']
    if s2:  dfList += ['progress_s2']
    for dfname in dfList:
        with r.statusLoad(dfname) as df:
            fluo = None
            fluoList = []
            for row in df.index:
                if fluo not in fluoList:
                    fluoList += [row[0]]

            for fluorophore in fluoList:
                df.loc[(fluorophore , functional), :] = None
            df.sort_index(inplace=True)

        if displayDF:
            prettyDisplay()

def removeFunctional() -> None:
    out.clear_output()
    global displayDF
    global functional
    global spectra
    global energy
    global gas
    global s0
    global s1
    global s2

    dfList = []
    if gas: dfList += ['progress_gas']
    if s0:  dfList += ['progress_s0']
    if s1:  dfList += ['progress_s1']
    if s2:  dfList += ['progress_s2']
    for dfname in dfList:
        with r.statusLoad(dfname) as df:
            try:
                df = df.drop(index=functional, level=1, inplace=True)
            except AttributeError:
                pass
                
        if displayDF:
            prettyDisplay()

def addFluorophore() -> None:
    out.clear_output()
    global displayDF
    global fluorophore
    global spectra
    global energy
    global qy
    global gas
    global ds
    global s0
    global s1
    global s2

    dfList = []
    if gas: dfList += ['progress_gas']
    if s0:  dfList += ['progress_s0']
    if s1:  dfList += ['progress_s1']
    if s2:  dfList += ['progress_s2']
    if ds:  dfList += ['spectra', 'dataset']
    if qy:  dfList += ['qy_ref']

    for dfname in dfList:
        with r.statusLoad(dfname) as df:
            # build list of methods
            fluo = None
            methodList = []
            for row in df.index:
                if fluo == None or fluo == row[0]:
                    fluo = row[0]
                    methodList += [row[1]]

            for method in methodList:
                df.loc[(fluorophore, method), :] = None
        if displayDF:
            prettyDisplay()

def removeFluorophore() -> None:
    out.clear_output()
    global displayDF
    global fluorophore
    global spectra
    global energy
    global qy
    global gas
    global s0
    global s1
    global s2
    global ds

    dfList = []
    if gas: dfList += ['progress_gas']
    if s0:  dfList += ['progress_s0']
    if s1:  dfList += ['progress_s1']
    if s2:  dfList += ['progress_s2']
    if ds:  dfList += ['spectra', 'dataset']
    if qy:  dfList += ['qy_ref']

    for dfname in dfList:
        with r.statusLoad(dfname) as df:
            # build list of methods
            fluo = None
            methodList = []
            for row in df.index:
                if fluo == None or fluo == row[0]:
                    fluo = row[0]
                    methodList += [row[1]]

            for method in methodList:
                df.drop(index=(fluorophore, method), inplace=True)
        if displayDF:
            prettyDisplay()

def addSolvent() -> None:
    out.clear_output()
    global displayDF
    global solvent
    global qy
    qy = True

    with r.statusLoad('qy_ref') as df:
        fluo = None
        fluoList = []
        for row in df.index:
            if fluo not in fluoList:
                fluoList += [row[0]]

        for fluorophore in fluoList:
            df.loc[(fluorophore, r.spectraType.qy), solvent] = None
        df.sort_index(inplace=True)

    if displayDF:
        prettyDisplay()

def removeSolvent() -> None:
    out.clear_output()
    global displayDF
    global solvent
    global qy
    qy = True

    with r.statusLoad('qy_ref') as df:
        df = df.drop(solvent, axis=1, inplace=True)

    if displayDF:
        prettyDisplay()

def viewAndPull() -> None:
    global displayDF
    global gas
    global s0
    global s1
    global s2
    out.clear_output()
    with out:
        with r.monarchHandler() as mon:
            if s0: 
                print('Pulling S0')
                check_s0(mon)
            if s1:
                print('Pulling S1')
                check_s1(mon)
            if s2:
                print('Pulling S2')
                check_s2(mon)
            if gas:
                print('Pulling Gas')
                check_gas(mon)
        
        if displayDF:
            prettyDisplay()

def viewAndPullWithCASSCF():
    global checkCASSCF
    checkCASSCF = True
    viewAndPull()

def viewAndPullNoCASSCF():
    global checkCASSCF
    checkCASSCF = False
    viewAndPull()


def timedOut() -> None:
    out.clear_output()
    with out:
        with r.monarchHandler() as mon:
            mon.timed_out()
            print('\n\n')

# Input handler
def readWidg(displayDF_in, fluorophore_in, functional_in, solvent_in, spectra_in,
             energy_in, qy_in, gas_in, s0_in, s1_in, s2_in, ds_in) -> None:
    global displayDF
    global fluorophore
    global solvent
    global functional
    global spectra
    global energy
    global qy
    global gas
    global ds
    global s0
    global s1
    global s2

    displayDF = displayDF_in
    fluorophore = fluorophore_in
    functional = functional_in
    spectra = spectra_in
    energy = energy_in
    solvent = solvent_in
    qy = qy_in
    gas = gas_in
    ds = ds_in
    s0 = s0_in
    s1 = s1_in
    s2 = s2_in
    return

reset_widg = widgets.interactive(resetDF, {'manual' : True, 'manual_name' : 'Reset DFs'})
add_func_widg = widgets.interactive(addFunctional, {'manual' : True, 'manual_name' : 'Add Functional'})
rem_func_widg = widgets.interactive(removeFunctional, {'manual' : True, 'manual_name' : 'Remove Functional'})
add_fluo_widg = widgets.interactive(addFluorophore, {'manual' : True, 'manual_name' : 'Add Fluorophore'})
rem_fluo_widg = widgets.interactive(removeFluorophore, {'manual' : True, 'manual_name' : 'Remove Fluorophore'})
add_solv_widg = widgets.interactive(addSolvent, {'manual' : True, 'manual_name' : 'Add Solvent to QY ref DB'})
rem_solv_widg = widgets.interactive(removeSolvent, {'manual' : True, 'manual_name' : 'Remove Solvent from QY ref DB'})

showDB_widg = widgets.interactive(showDF, {'manual' : True, 'manual_name' : 'Show Selected Dataframes'})
showQYDB_widg = widgets.interactive(showQYDF, {'manual' : True, 'manual_name' : 'Show QY Dataframe'})
showALL_widg = widgets.interactive(showALL, {'manual' : True, 'manual_name' : 'Show ALL Dataframes'})
viewPull_widg = widgets.interactive(viewAndPullWithCASSCF, {'manual' : True, 'manual_name' : 'View and Pull'})
viewPull_NOC_widg = widgets.interactive(viewAndPullNoCASSCF, {'manual' : True, 'manual_name' : 'View and Pull Without CASSCF'})
to_widg = widgets.interactive(timedOut, {'manual' : True, 'manual_name' : 'Print Timed-Out Jobs'})
vis_widg = widgets.interactive(visualise_ds, {'manual' : True, 'manual_name' : 'Visualise Dataset'})

displayDF_widg = widgets.Checkbox(value=True, description='Show the modified DBs', layout=layout)
fluorophore_widg = widgets.Dropdown(options=r.Fluorophores, description='Fluorophore', layout=layout)
functional_widg = widgets.Dropdown(options=r.Methods, description='Method', layout=layout)
solvent_widg = widgets.Dropdown(options=r.Solvents, description='Solvent', layout=layout)
gas_widg = widgets.Checkbox(value=False, description='Gas DB', layout=layout)
ds_widg = widgets.Checkbox(value=False, description='Dataset', layout=layout)
s0_widg = widgets.Checkbox(value=False, description='S0 DB', layout=layout)
s1_widg = widgets.Checkbox(value=False, description='S1 DB', layout=layout)
s2_widg = widgets.Checkbox(value=False, description='S2 DB', layout=layout)
spectra_widg = widgets.Checkbox(value=False, description='Spectra DB', layout=layout)
energy_widg = widgets.Checkbox(value=False, description='Energy Allcation DB', layout=layout)
qy_widg = widgets.Checkbox(value=False, description='Quantum Yield Reference DB', layout=layout)

widg_loader = widgets.interactive_output(readWidg, {'displayDF_in': displayDF_widg,
                                                    'fluorophore_in': fluorophore_widg,
                                                    'functional_in': functional_widg,
                                                    'solvent_in': solvent_widg,
                                                    'spectra_in': spectra_widg,
                                                    'energy_in': energy_widg,
                                                    'qy_in': qy_widg,
                                                    'gas_in': gas_widg,
                                                    's0_in': s0_widg,
                                                    's1_in': s1_widg,
                                                    's2_in': s2_widg,
                                                    'ds_in': ds_widg,
                                                    })
view = widgets.VBox([displayDF_widg, spectra_widg, energy_widg, qy_widg, gas_widg, s0_widg, s1_widg, s2_widg, to_widg, vis_widg, widgets.HBox([viewPull_widg, viewPull_NOC_widg]), widgets.HBox([showDB_widg, showALL_widg])])
reset = widgets.VBox([displayDF_widg,spectra_widg, energy_widg, qy_widg, gas_widg, s0_widg, s1_widg, s2_widg, reset_widg, showDB_widg])
func = widgets.VBox([displayDF_widg, functional_widg, gas_widg, s0_widg, s1_widg, s2_widg, add_func_widg, rem_func_widg, showDB_widg])
fluo = widgets.VBox([displayDF_widg, fluorophore_widg, ds_widg, qy_widg, gas_widg, s0_widg, s1_widg, s2_widg, add_fluo_widg, rem_fluo_widg,])
solv = widgets.VBox([displayDF_widg, solvent_widg, add_solv_widg, rem_solv_widg, showQYDB_widg])

children = [view, reset, func, fluo, solv]
tab = widgets.Tab()
tab.children = children
tab.set_title(0, 'General')
tab.set_title(1, 'Reset DFs')
tab.set_title(2, 'Add/Remove Methods')
tab.set_title(3, 'Add/Remove Fluorophores')
tab.set_title(4, 'Add/Remove Solvent')
display(tab)

out = widgets.Output()

display(widgets.VBox([widg_loader, out]))